# Métricas e Resultados

In [1]:
import numpy as np
import ast
import json
import pandas as pd

In [12]:
#Função que retorna o elemento mais frequente e sua frequencia
def maisFrequente(grupo,frequencias):
    maior = 0
    resp = ""
    for g in grupo:
        if(frequencias[g] > maior):
            maior = frequencias[g]
            resp = g
    return resp,maior

def read_json(source_file):
    with open(source_file, 'r') as fp:
        data = json.load(fp)
    return data

def read_gabarito(source_file):
    gold = {}
    arq = open(source_file, 'r')
    for linha in arq.readlines():
        s = linha.split(':')
        classe = s[0]
        results = ast.literal_eval(s[1])
        gold[classe] = set(results) #Mapeando as classes com suas palavras
    arq.close()
    
    return gold

def prob_groups(result, gold):
    results = []
    percent = []
    
    for cl in result:
        p = {}
        for g in sorted(gold.keys()):
            corrects = len(gold[g].intersection(set(cl)))
            total = len(gold[g])
            rec = corrects/total
            p[g] = rec
            
        percent.append(p)
        a = np.array(list(p.keys()))
        b = np.array(list(p.values()))
        pos = np.argmax(b)
        results.append((a[pos], b[pos]))
        
    return percent, results
    
def prob_rename_groups(percent, result, metricas_atual, respostas):
    resps = []
    data_group = {}
    with open('./metricas/alternativos/' + metricas_atual, 'w') as fp:
        for i, pct in enumerate(percent):
            data_group['Group' + str(i)] = {}
            data_group['Group' + str(i)]['word'] = result[i]
            list_perc = []
            for p in pct.keys():
                pc = (p, pct[p])
                list_perc.append(pc)
            data_group['Group' + str(i)]['class'] = list_perc
            data_group['Group' + str(i)]['result'] = respostas[i]
            
        json.dump(data_group, fp)
        
def freq_values(result, data_freq_atual, gold):
    resps = []
    for idx, value in enumerate(result):
        element, val = maisFrequente(value, data_freq_atual)
        
        for g in gold.keys():
            if element in gold[g]:
                resps.append((element, g))
                break
                
    return resps

def prob_groups_rename(metricas_frequencia_atual, resps):
    with open('./metricas/alternativos/' + metricas_frequencia_atual, 'w') as fp:
        data = {}
        lista_repre = []
        for idx, r in enumerate(resps):
            lista_repre.append((r[0], r[1]))
        data['representante'] = lista_repre
        json.dump(data, fp)

In [17]:
gabarito = ['DadosCamera.txt', 'DadosCells.txt','DadosDvds.txt','DadosLaptops.txt','DadosRouters.txt'] #Diretorios dos gabaritos
resultado_alternativo = ['agrupamentos-finais-camera-alternativo.json','agrupamentos-finais-cells-alternativo.json','agrupamentos-finais-dvds-alternativo.json','agrupamentos-finais-laptops-alternativo.json','agrupamentos-finais-routers-alternativo.json'] #Diretorios dos agrupamentos
freq = ['frequencia-camera.json','frequencia-cells.json','frequencia-dvds.json','frequencia-laptops.json','frequencia-routers.json'] #Diretorios das frequencias

In [4]:
metricas_alternativo = ['metricas-camera.json', 'metricas-cells.json', 'metricas-dvds.json', 'metricas-laptops.json', 'metricas-routers.json']

metricas_frequencia_alternativo = ['metricas-camera-frequencia.json', 'metricas-cells-frequencia.json', 'metricas-dvds-frequencia.json', 'metricas-laptops-frequencia.json', 'metricas-routers-frequencia.json']

metricas_resultado_alternativo = ['metricas-resultado-camera.txt', 'metricas-resultado-cells.txt', 'metricas-resultado-dvds.txt', 'metricas-resultado-laptops.txt', 'metricas-resultado-routers.txt']

metricas_agrupamento_alternativo = ['agrupamentos-camera-resultado.json', 'agrupamentos-cells-resultado.json', 'agrupamentos-dvds-resultado.json', 'agrupamentos-laptops-resultado.json', 'agrupamentos-routers-resultado.json']

save_resultados_alternativo = ['resultados-camera.csv', 'resultados-cells.csv', 'resultados-dvds.csv', 'resultados-laptops.csv', 'resultados-routers.csv']

k = [9,10,6,12,10]

In [18]:
for idx, value in enumerate(gabarito):
    precision = []
    recall = []
    f1_score = []
    
    gabarito_atual = gabarito[idx]
    resultado_atual = resultado_alternativo[idx]
    freq_atual = freq[idx]
    
    metricas_atual = metricas_alternativo[idx]
    metricas_frequencia_atual = metricas_frequencia_alternativo[idx]
    metricas_resultado_atual = metricas_resultado_alternativo[idx]
    metricas_agrupamento_atual = metricas_agrupamento_alternativo[idx]
    save_atual = save_resultados_alternativo[idx]
    
    data_gabarito = read_gabarito('../datasets/gabaritos/' + gabarito_atual)
    data_result_atual = read_json('./agrupamentos/' + resultado_atual)
    
    # listando os agrupamentos gerados
    contador = 0
    result = []
    for key in data_result_atual.keys():
        if contador < k[idx]:
            if len(data_result_atual[key]) > 2:
                result.append(data_result_atual[key])
                contador += 1
        else:
            break
    
    # obtendo as frequencias das palavras
    data_freq_atual = read_json('../datasets_processed/frequencias/' + freq_atual)

    
    # calculando as probabilidades para os nomes dos grupos
    percent, respostas = prob_groups(result, data_gabarito)
    
    prob_rename_groups(percent, result, metricas_atual, respostas)
    
    resps = freq_values(result, data_freq_atual, data_gabarito)
    
    # nomeando só grupos baseados no termo mais frequente do grupo
    prob_groups_rename(metricas_frequencia_atual, resps)
    
    #Salvando os resultados das metricas por grupo
    arq = open('./metricas/alternativos/' + metricas_resultado_atual,'w')
    for i in range(len(respostas)):
        arq.write(str(result[i]) + '\n\n')
        key,valor = respostas[i][0],respostas[i][1]
        agree = len(set(result[i]).intersection(data_gabarito[key]))
        precisao = agree/len(result[i])
        revocacao = agree/len(data_gabarito[key])
        f1 = (2*precisao*revocacao)/(precisao+revocacao)
        precision.append(precisao)
        recall.append(revocacao)
        f1_score.append(f1)
        arq.write('Classe: ' + key + ' (' + str(valor) + ')\n\n')
        arq.write('Precisao: ' + str(precisao) + '\n\n')
        arq.write('Revocacao: ' + str(revocacao) + '\n\n')
        arq.write('F1: ' + str(f1) + '\n\n\n')
    arq.close()
    
    dados = np.array([precision,recall,f1_score])
    dados = np.transpose(dados)
    df = pd.DataFrame(dados,columns=["Precisao","Revocacao","F1"])
    
    df["Classe"] = [ans[0] for ans in respostas]
    df["Probabilidade"] = [ans[1] for ans in respostas]
    df["Grupos"] = ["Grupo "+str(ind+1) for ind in range(len(respostas))]
    df = df.set_index("Grupos")
    
    df.to_csv(save_atual)

In [21]:
df = pd.read_csv('resultados-camera.csv')
df

,Grupos,Precisao,Revocacao,F1,Classe,Probabilidade
0,Grupo 1,0.363636,0.500000,0.421053,Imaging,0.500000
1,Grupo 2,0.909091,0.125000,0.219780,Others,0.125000
2,Grupo 3,0.416667,0.312500,0.357143,Imaging,0.312500
3,Grupo 4,0.222222,0.444444,0.296296,Zoom,0.444444
4,Grupo 5,0.333333,0.333333,0.333333,Zoom,0.333333
5,Grupo 6,0.230769,0.750000,0.352941,Performance,0.750000
6,Grupo 7,0.428571,0.600000,0.500000,Price,0.600000
7,Grupo 8,0.181818,0.500000,0.266667,Performance,0.500000
8,Grupo 9,1.000000,0.037500,0.072289,Others,0.037500
